In [22]:
import requests
import json
import itertools
import os
import datetime
import pickle
import time
import random
import pandas as pd
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from tqdm.notebook import tqdm

# --- Configuration ---
REPO_OWNER = "JabRef"
REPO_NAME = "jabref"
REPOSITORY = f"{REPO_OWNER}/{REPO_NAME}"

# Date format: YYYY-MM-DDTHH:MM (e.g. 2025-07-31T00:00)
# Start date for the mining process
START_DATE = "2025-07-31T00:00"
# End date (Optional). Set to None to use the current time.
END_DATE = "2025-08-01T00:00" 

# File paths
# Assuming this notebook is in /mining, and env is in the root
TOKENS_FILE = "../env/tokens.txt" 

# Output directory
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

STATE_FILE = os.path.join(OUTPUT_DIR, f"{REPO_NAME}_mining_state.pkl")
LOG_FILE = os.path.join(OUTPUT_DIR, f"{REPO_NAME}_mining.log")
OUTPUT_CSV = os.path.join(OUTPUT_DIR, f"{REPO_NAME}_prs.csv")
OUTPUT_JSON = os.path.join(OUTPUT_DIR, f"{REPO_NAME}_prs.json")

# Tuning
DEFAULT_DAYS_INTERVAL = 60
MAX_RESULTS_PER_QUERY = 1000 # GitHub limit is 1000
SAFETY_THRESHOLD = 950 # Reduce interval if results > this

In [23]:
# --- Token Management ---

def load_tokens(filepath):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Tokens file not found at {filepath}")
    with open(filepath, "r") as file:
        tokens = [t.strip() for t in file.read().splitlines() if t.strip()]
    if not tokens:
        raise ValueError("No tokens found in file.")
    return tokens

tokens = load_tokens(TOKENS_FILE)
token_iterator = itertools.cycle(tokens)
current_token = next(token_iterator)

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
]

def get_headers(token):
    return {
        "Authorization": f"Bearer {token}",
        "User-Agent": random.choice(user_agents),
    }

# Setup GraphQL client
graphql_url = "https://api.github.com/graphql"
transport = RequestsHTTPTransport(url=graphql_url, headers=get_headers(current_token), use_json=True)
client = Client(transport=transport, fetch_schema_from_transport=True)

def log_activity(activity: str):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] {activity}\n"
    print(activity)
    with open(LOG_FILE, "a") as log_file:
        log_file.write(log_entry)

log_activity(f"Loaded {len(tokens)} tokens.")

Loaded 3 tokens.


In [24]:
# --- GraphQL Queries ---

# Rate limit query
rate_limit_query = gql(
    """
    query {
      rateLimit {
        limit
        remaining
        resetAt
      }
    }
    """
)

# Main PR query
query_template = gql(
    """
    query searchIssues($keyword: String!, $afterCursor: String, $first: Int) {
      search(query: $keyword, type: ISSUE, first: $first, after: $afterCursor) {
        issueCount
        edges {
          cursor
          node {
            ... on PullRequest {
              id
              number
              title
              url
              comments {
                totalCount
              }
              state
              closed
              merged
              createdAt
              updatedAt
              mergeCommit {
                oid
              }
              timeline(last: 100) {
                edges {
                  node {
                    __typename
                    ... on ClosedEvent { 
                      actor {   
                        ... on User { login url }
                        ... on Mannequin { login url }
                        ... on Bot { login url }
                      }
                      createdAt
                    }
                    ... on MergedEvent { 
                      actor {   
                        ... on User { login url }
                        ... on Mannequin { login url }
                        ... on Bot { login url }
                      }
                      createdAt
                    }
                  }
                }
              }
              commits {
                totalCount
              }
              changedFiles
              additions
              deletions
              headRefName
              baseRefName
              repository {
                id
                nameWithOwner
                stargazerCount
                description
                isFork
                languages(first: 20) {
                  edges {
                    node {
                      name
                    }
                  }
                }
              }
              author {
                 ... on User { login url }
                 ... on Mannequin { login url }
                 ... on Bot { login url }
              }
              labels(first: 20) {
                edges {
                  node {
                    name
                  }
                }
              }
              body
            }
          }
        }
        pageInfo {
          endCursor
          hasNextPage
        }
      }
    }
    """
)

In [ ]:
# --- Helper Functions ---

def check_and_rotate_token():
    global current_token, transport, client
    
    try:
        # Check current token
        transport.headers = get_headers(current_token)
        response = client.execute(rate_limit_query)
        remaining = response["rateLimit"]["remaining"]
        
        if remaining < 100:
            log_activity(f"Token rate limit low ({remaining}). Rotating...")
            
            # Try finding a good token
            initial_token = current_token
            while True:
                current_token = next(token_iterator)
                transport.headers = get_headers(current_token)
                
                try:
                    resp = client.execute(rate_limit_query)
                    rem = resp["rateLimit"]["remaining"]
                    if rem >= 100:
                        log_activity(f"Switched to token with {rem} remaining.")
                        return
                except Exception as e:
                    if "Bad credentials" in str(e) or "401" in str(e):
                         log_activity(f"Token invalid (skipped): {current_token[:10]}...")
                    pass # Token might be bad, skip
                
                if current_token == initial_token:
                    log_activity("All tokens exhausted. Sleeping for 30 minutes...")
                    time.sleep(1800)
                    
    except Exception as e:
        if "Bad credentials" in str(e) or "401" in str(e):
            log_activity(f"Current token invalid: {e}. Rotating...")
        else:
            log_activity(f"Error checking rate limit: {e}. Rotating token.")
        
        current_token = next(token_iterator)
        transport.headers = get_headers(current_token)

def execute_query_safe(keyword, first=100, after_cursor=None):
    """Executes query with retries and token rotation."""
    global current_token
    
    retries = 0
    max_retries = 5
    
    while retries < max_retries:
        try:
            check_and_rotate_token()
            
            return client.execute(
                query_template,
                variable_values={
                    "keyword": keyword,
                    "first": first,
                    "afterCursor": after_cursor,
                },
            )
        except Exception as e:
            log_activity(f"Query failed: {e}")
            if "API rate limit" in str(e) or "502" in str(e) or "500" in str(e) or "Bad credentials" in str(e) or "401" in str(e):
                log_activity("Rate limit, auth error, or server error. Rotating/Sleeping...")
                current_token = next(token_iterator)
                time.sleep(5)
            else:
                # Reduce page size if it's a complexity issue
                if first > 10:
                    first = int(first / 2)
                    log_activity(f"Reducing page size to {first}")
                else:
                    log_activity("Persistent error. Skipping...")
                    raise e
            retries += 1
            
    raise Exception("Max retries reached")

def process_pr_node(node, repository_name):
    """Extracts relevant fields from the raw GraphQL node."""
    if not node: return None
    
    timeline = node.get("timeline", {}).get("edges", [])
    
    # Helper to find event
    def find_event(typename):
        return next((x["node"] for x in timeline if x["node"] and x["node"]["__typename"] == typename), None)

    closed_event = find_event("ClosedEvent")
    merged_event = find_event("MergedEvent")
    
    author = node.get("author") or {}
    repo = node.get("repository") or {}
    
    return {
        "id": node.get("id"),
        "pull_number": node.get("number"),
        "title": node.get("title"),
        "url": node.get("url"),
        "state": node.get("state"),
        "comments_count": node.get("comments", {}).get("totalCount", 0),
        "closed": node.get("closed"),
        "closed_by": closed_event.get("actor", {}).get("login") if closed_event and closed_event.get("actor") else None,
        "closed_at": closed_event.get("createdAt") if closed_event else None,
        "merged": node.get("merged"),
        "merged_by": merged_event.get("actor", {}).get("login") if merged_event and merged_event.get("actor") else None,
        "merged_at": merged_event.get("createdAt") if merged_event else None,
        "body": node.get("body"),
        "created_at": node.get("createdAt"),
        "updated_at": node.get("updatedAt"),
        "repository_name_with_owner": repo.get("nameWithOwner"),
        "repository_stargazer_count": repo.get("stargazerCount"),
        "repository_is_fork": repo.get("isFork"),
        "repository_languages": [l["node"]["name"] for l in repo.get("languages", {}).get("edges", []) if l.get("node")],
        "merge_commit_oid": node.get("mergeCommit", {}).get("oid") if node.get("mergeCommit") else None,
        "labels": [l["node"]["name"] for l in node.get("labels", {}).get("edges", []) if l.get("node")],
        "commits_count": node.get("commits", {}).get("totalCount", 0),
        "changed_files_count": node.get("changedFiles"),
        "additions_count": node.get("additions"),
        "deletions_count": node.get("deletions"),
        "author_name": author.get("login"),
        "author_url": author.get("url"),
        "search_repository": repository_name,
    }

In [26]:
# --- State Management ---

def load_state():
    if os.path.exists(STATE_FILE):
        try:
            with open(STATE_FILE, "rb") as f:
                state = pickle.load(f)
            log_activity(f"Resuming from state. Collected {len(state.get('data', []))} PRs so far.")
            log_activity(f"Last processed date: {state.get('last_date')}")
            return state
        except Exception as e:
            log_activity(f"Error loading state: {e}. Starting fresh.")
    
    return {
        "data": [],
        "last_date": START_DATE,
        "params": {"repo": REPOSITORY}
    }

def save_state(data, last_date):
    state = {
        "data": data,
        "last_date": last_date,
        "params": {"repo": REPOSITORY},
        "updated_at": datetime.datetime.now().isoformat()
    }
    with open(STATE_FILE, "wb") as f:
        pickle.dump(state, f)

In [27]:
# --- Main Mining Loop ---

def mine_prs():
    state = load_state()
    all_prs = state["data"]
    
    # Determine start and end times
    current_start_str = state["last_date"]
    start_dt = datetime.datetime.strptime(current_start_str, "%Y-%m-%dT%H:%M")
    
    if END_DATE:
        end_dt = datetime.datetime.strptime(END_DATE, "%Y-%m-%dT%H:%M")
    else:
        end_dt = datetime.datetime.now()
        
    days_interval = DEFAULT_DAYS_INTERVAL
    
    log_activity(f"Starting mining from {start_dt} to {end_dt}")
    
    while start_dt < end_dt:
        # Calculate next date
        next_dt = start_dt + datetime.timedelta(days=days_interval)
        if next_dt > end_dt:
            next_dt = end_dt
            
        date_range_str = f"{start_dt.strftime('%Y-%m-%dT%H:%M')}..{next_dt.strftime('%Y-%m-%dT%H:%M')}"
        search_query = f"repo:{REPOSITORY} is:pr is:public archived:false created:{date_range_str}"
        
        log_activity(f"Fetching range: {date_range_str}")
        
        try:
            # Fetch first page to check count
            response = execute_query_safe(search_query, first=1)
            total_count = response["search"]["issueCount"]
            log_activity(f"Found {total_count} PRs in range.")
            
            # Adjust interval if too many results
            if total_count > SAFETY_THRESHOLD:
                new_interval = max(1, days_interval // 2)
                if new_interval < days_interval:
                    log_activity(f"Too many results ({total_count}). Reducing interval from {days_interval} to {new_interval} days.")
                    days_interval = new_interval
                    continue # Retry with smaller interval
            
            # If count is good, fetch all pages
            after_cursor = None
            page_count = 0
            
            while True:
                response = execute_query_safe(search_query, first=100, after_cursor=after_cursor)
                edges = response["search"]["edges"]
                
                if not edges:
                    break
                    
                for edge in edges:
                    pr_data = process_pr_node(edge["node"], REPOSITORY)
                    if pr_data:
                        all_prs.append(pr_data)
                
                page_info = response["search"]["pageInfo"]
                if page_info["hasNextPage"]:
                    after_cursor = page_info["endCursor"]
                    page_count += 1
                else:
                    break
            
            # Successful interval
            # Save state
            save_state(all_prs, next_dt.strftime('%Y-%m-%dT%H:%M'))
            
            # Advance time
            start_dt = next_dt
            
            # Try to increase interval back to default if it was reduced
            if days_interval < DEFAULT_DAYS_INTERVAL:
                days_interval = min(DEFAULT_DAYS_INTERVAL, days_interval * 2)
                
        except Exception as e:
            log_activity(f"Critical error in loop: {e}")
            time.sleep(60) # Wait a bit before retrying same block or manual intervention
            # We do NOT advance start_dt here, so it retries the same block next time
            raise e

    log_activity("Mining completed.")
    return all_prs

# Run the miner
collected_data = mine_prs()

Starting mining from 2025-07-31 00:00:00 to 2025-08-01 00:00:00
Fetching range: 2025-07-31T00:00..2025-08-01T00:00
Found 2 PRs in range.
Found 2 PRs in range.
Mining completed.
Mining completed.


In [28]:
# --- Post-Processing & Saving ---

def save_results(data):
    if not data:
        log_activity("No data to save.")
        return

    df = pd.DataFrame(data)
    
    # Deduplicate by ID
    initial_len = len(df)
    df = df.drop_duplicates(subset="id", keep="first")
    log_activity(f"Deduplicated: {initial_len} -> {len(df)} records.")
    
    # Save CSV
    df.to_csv(OUTPUT_CSV, index=False)
    log_activity(f"Saved to {OUTPUT_CSV}")
    
    # Save JSON
    df.to_json(OUTPUT_JSON, orient="records", indent=4)
    log_activity(f"Saved to {OUTPUT_JSON}")

save_results(collected_data)

Deduplicated: 2 -> 2 records.
Saved to outputs/jabref_prs.csv
Saved to outputs/jabref_prs.json
